In [1]:
from WindPy import w

In [2]:
import pandas as pd
import numpy as np
from WindPy import w
from scipy.optimize import minimize
import datetime

# Start WindPy
w.start()
tickers = ['HSI.HI', 'SPX.GI']
start_date = '2000-01-01'
end_date = '2023-12-31'
risk_free_rate = 0.025

market_data = w.wsd(tickers, "close", start_date, end_date)
prices = pd.DataFrame({code: market_data.Data[i] for i, code in enumerate(tickers)}, index=market_data.Times)
prices.dropna(inplace=True)

returns = prices.pct_change().dropna()
fed_data = w.wsd("EFFR.IR", "close", start_date, end_date)
fed_rate = pd.Series(fed_data.Data[0], index=fed_data.Times).dropna()
fed_diff = fed_rate.diff()
direction = fed_diff.apply(lambda x: 'rising' if x > 0 else ('falling' if x < 0 else 'flat'))

returns['Rate_Direction'] = direction
rising_returns = returns[returns['Rate_Direction'] == 'rising'].drop(columns='Rate_Direction')
falling_returns = returns[returns['Rate_Direction'] == 'falling'].drop(columns='Rate_Direction')
def calculate_portfolio_stats(weights, returns):
    ret = np.sum(returns.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    return ret, vol

def negative_sharpe_ratio(weights, returns, risk_free_rate):
    ret, vol = calculate_portfolio_stats(weights, returns)
    return -((ret - risk_free_rate) / vol)

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(2))
initial_guess = [0.5, 0.5]
# Rising period optimization
opt_rising = minimize(negative_sharpe_ratio, initial_guess, args=(rising_returns, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)
rising_weights = opt_rising.x
rising_ret, rising_vol = calculate_portfolio_stats(rising_weights, rising_returns)

# Falling period optimization
opt_falling = minimize(negative_sharpe_ratio, initial_guess, args=(falling_returns, risk_free_rate),
                       method='SLSQP', bounds=bounds, constraints=constraints)
falling_weights = opt_falling.x
falling_ret, falling_vol = calculate_portfolio_stats(falling_weights, falling_returns)
print("=== Rising Interest Rate Period ===")
print(f"HSI weight: {rising_weights[0]*100:.2f}%")
print(f"SPX weight: {rising_weights[1]*100:.2f}%")
print(f"Expected Return: {rising_ret:.2f}")
print(f"Volatility: {rising_vol:.2f}")
print(f"Sharpe Ratio: {(rising_ret - risk_free_rate)/rising_vol:.2f}")

print("\n=== Falling Interest Rate Period ===")
print(f"HSI weight: {falling_weights[0]*100:.2f}%")
print(f"SPX weight: {falling_weights[1]*100:.2f}%")
print(f"Expected Return: {falling_ret:.2f}")
print(f"Volatility: {falling_vol:.2f}")
print(f"Sharpe Ratio: {(falling_ret - risk_free_rate)/falling_vol:.2f}")

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
=== Rising Interest Rate Period ===
HSI weight: 0.00%
SPX weight: 100.00%
Expected Return: -0.07
Volatility: 0.22
Sharpe Ratio: -0.44

=== Falling Interest Rate Period ===
HSI weight: 100.00%
SPX weight: 0.00%
Expected Return: 0.17
Volatility: 0.27
Sharpe Ratio: 0.55
